In [ ]:
! ./udpipe-1.2.0 --tokenize --tag --parse bosque-ud-2.6.udpipe dhbb.raw > dhbb.conllu
! ./udpipe-1.2.0 --tokenize --tag --parse bosque-ud-2.6.udpipe obras.raw > obras.conllu

In [1]:
# Definindo variáveis iniciais
import estrutura_ud
import os

root_deprel = ["root"]
subordinate_deprel = ["acl:relcl", "ccomp", "advcl"]
max_examples = 20

path = {
    'bosque': "bosque-ud-2.6.conllu",
    'dhbb': "dhbb.conllu",
    "obras": "obras.conllu"
}

In [2]:
# Carregamento dos corpora (demora muito!)
bosque = estrutura_ud.Corpus(recursivo=True)
dhbb = estrutura_ud.Corpus(recursivo=True)
obras = estrutura_ud.Corpus(recursivo=True)
bosque.load(path["bosque"])
dhbb.load(path["dhbb"])
obras.load(path["obras"])

corpora = {
    'dhbb': dhbb,
    'obras': obras,
    'bosque': bosque
}

In [3]:
# Estatísticas dos corpora
stats_corpora = {}
for corpus in corpora:
    stats_corpora[corpus] = {
        "size": os.path.getsize(path[corpus])/1024, 
        "n_tokens": 0, 
        "n_sentences": 0,
    }
    
for corpus in corpora:
    for sentence in corpora[corpus].sentences.values():
        stats_corpora[corpus]['n_sentences'] += 1
        for token in sentence.tokens:
            if not '-' in token.id:
                stats_corpora[corpus]['n_tokens'] += 1
    
print(stats_corpora)

{'dhbb': {'size': 960964.294921875, 'n_tokens': 16037286, 'n_sentences': 480218}, 'obras': {'size': 480188.513671875, 'n_tokens': 7863261, 'n_sentences': 353662}, 'bosque': {'size': 14846.041015625, 'n_tokens': 227825, 'n_sentences': 9364}}


In [4]:
# Números de orações
total_deprel = {}
for corpus in corpora:
    total_deprel[corpus] = {
        "|".join(root_deprel): 0,
        "|".join(subordinate_deprel): 0
    }
    for sentence in corpora[corpus].sentences.values():
        for token in sentence.tokens:
            if token.deprel in root_deprel:
                total_deprel[corpus]["|".join(root_deprel)] += 1
            if token.deprel in subordinate_deprel:
                total_deprel[corpus]["|".join(subordinate_deprel)] += 1
print(total_deprel)

{'dhbb': {'root': 480218, 'acl:relcl|ccomp|advcl': 341133}, 'obras': {'root': 353662, 'acl:relcl|ccomp|advcl': 316297}, 'bosque': {'root': 9364, 'acl:relcl|ccomp|advcl': 6842}}


# Busca ingênua tanto em root quanto em subordinada

In [5]:
def find_clause_without_subject(sentences, deprel_list):
    clauses_without_subject = {}
    for sent_id, sentence in sentences.items():
        for t, token in enumerate(sentence.tokens):
            if token.deprel in deprel_list:
                has_subj = False
                for _token in sentence.tokens:
                    if _token.dephead == token.id and _token.deprel in ["nsubj", "csubj", "nsubj:pass"]:
                        has_subj = True
                        break
                if not has_subj:
                    if not sent_id in clauses_without_subject:
                        clauses_without_subject[sent_id] = []
                    clauses_without_subject[sent_id].append(t)
    return clauses_without_subject

root_without_subject = {}
subordinate_without_subject = {}
for corpus in corpora:
    root_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, root_deprel)
    subordinate_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, subordinate_deprel)

In [6]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto ingênuo: 226122 / 480218
dhbb - orações subordinadas com sujeito oculto ingênuo: 190381 / 341133

obras - orações principais com sujeito oculto ingênuo: 172124 / 353662
obras - orações subordinadas com sujeito oculto ingênuo: 156040 / 316297

bosque - orações principais com sujeito oculto ingênuo: 2777 / 9364
bosque - orações subordinadas com sujeito oculto ingênuo: 2617 / 6842

=== 20 exemplos de sujeito oculto na oração principal em dhbb ===
2: *Estudou* em o Ginásio Diocesano de São Paulo e bacharelou se em 1910 por a Faculdade de Ciências Jurídicas e Sociais .
3: Dedicando se a a advocacia , foi *promotor* público em Cunha ( SP ) e depois delegado de polícia em o Rio de Janeiro , então Distrito Federal .
4: *Iniciou* sua vida política como deputado federal por o Distrito Federal , exercendo o mandato de 1927 a 1929 .
6: Ligado a o governo federal , *encontrava* se a o lado de o presidente Washington Luís , em o palácio Guanabara , em o 

# Filtro A

In [7]:
def find_haver_impessoal(sentences, list_tokens):
    haver_impessoal = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma == "haver" and "Number=Sing" in token.feats and "Person=3" in token.feats:
                if not sent_id in haver_impessoal:
                    haver_impessoal[sent_id] = []
                haver_impessoal[sent_id].append(t)
    return haver_impessoal
                
root_haver_impessoal = {}
subordinate_haver_impessoal = {}
for corpus in corpora:
    root_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_haver_impessoal[corpus]:
        for token in root_haver_impessoal[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_haver_impessoal[corpus]:
        for token in subordinate_haver_impessoal[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [9]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in root_haver_impessoal[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in subordinate_haver_impessoal[corpus].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de haver impessoal na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_haver_impessoal[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de haver impessoal na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_haver_impessoal[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto e haver impessoal: 1083
dhbb - orações subordinadas com sujeito oculto e haver impessoal: 861

obras - orações principais com sujeito oculto e haver impessoal: 5181
obras - orações subordinadas com sujeito oculto e haver impessoal: 4395

bosque - orações principais com sujeito oculto e haver impessoal: 124
bosque - orações subordinadas com sujeito oculto e haver impessoal: 148

=== 20 exemplos de haver impessoal na oração principal em dhbb ===
486: *Há* entretanto quem julgue , como Alexandre Barbosa Lima Sobrinho , que suas realizações foram motivadas por o afã de projetar nacionalmente o próprio nome de Antônio Carlos , tendo em vista a sucessão de Washington Luís , que assumira a presidência de a República em 15 de novembro de 1926 .
553: *Houve* idas e vindas a propósito de a data em que seria possível deflagrar a luta armada .
639: *Houve* críticas , formuladas , segundo Helena Bomeny , « a partir de o pressuposto de que o pacto entre 

# Filtro B

In [10]:
def find_nominals(sentences, list_tokens):
    nominals = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.upos != "VERB":
                is_cop = False
                for _token in sentence.tokens:
                    if _token.head_token.id == token.id and _token.deprel == "cop":
                        is_cop = True
                        break
                if not is_cop:
                    if not sent_id in nominals:
                        nominals[sent_id] = []
                    nominals[sent_id].append(t)
    return nominals
                
root_nominals = {}
for corpus in corpora:
    root_nominals[corpus] = find_nominals(corpora[corpus].sentences, root_without_subject[corpus])
    
    for sentence in root_nominals[corpus]:
        for token in root_nominals[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)

In [11]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e nominal: {}".format(corpus, sum([len(x) for x in root_nominals[corpus].values()])))

for corpus in corpora:
    i = 0
    print("\n=== {} exemplos de nominais como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_nominals[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break


dhbb - orações principais com sujeito oculto e nominal: 31599

obras - orações principais com sujeito oculto e nominal: 43326

bosque - orações principais com sujeito oculto e nominal: 1145

=== 20 exemplos de nominais como oração principal em dhbb ===
28: *Deputados* em fevereiro seguinte .
110: As *Oposições*
117: *Magalhães* , o voto de os representantes classistas e de os republicanos paulistas e ainda o beneplácito de Valadares .
120: *Transcorreria* em 1937 a campanha para a sucessão presidencial .
133: *Pinto* , visando promover uma manifestação de personalidades que contestavam o Estado Novo .
138: « *Manifesto* de os mineiros » , primeiro pronunciamento público de setores liberais contra o Estado Novo .
148: Aleixo fora *um* de os que propuseram o nome de Eduardo Gomes para as eleições presidenciais de 2 de dezembro de 1945 .
150: Dutra , *ministro* de a Guerra de Vargas , tendo como defensores Benedito Valadares e vários outros políticos ligados a o Estado Novo .
157: *Dutra

# Filtro C

In [12]:
fenomenos_natureza = "chover|ventar|nevar|chuviscar|garoar|gear|anoitecer|amanhecer|entardecer|relampejar|trovejar".split("|")
def find_natureza(sentences, list_tokens):
    natureza = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma in fenomenos_natureza:
                if not sent_id in natureza:
                    natureza[sent_id] = []
                natureza[sent_id].append(t)
    return natureza
                
root_natureza = {}
subordinate_natureza = {}
for corpus in corpora:
    root_natureza[corpus] = find_natureza(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_natureza[corpus] = find_natureza(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_natureza[corpus]:
        for token in root_natureza[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_natureza[corpus]:
        for token in subordinate_natureza[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [13]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in root_natureza[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in subordinate_natureza[corpus].values()])))
    
for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de fenômeno natureza como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_natureza[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de fenômeno natureza como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_natureza[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break


dhbb - orações principais com sujeito oculto e fenômeno natureza: 0
dhbb - orações subordinadas com sujeito oculto e fenômeno natureza: 4

obras - orações principais com sujeito oculto e fenômeno natureza: 100
obras - orações subordinadas com sujeito oculto e fenômeno natureza: 127

bosque - orações principais com sujeito oculto e fenômeno natureza: 1
bosque - orações subordinadas com sujeito oculto e fenômeno natureza: 2

=== 20 exemplos de fenômeno natureza como oração principal em dhbb ===

=== 20 exemplos de fenômeno natureza como oração subordinada em dhbb ===
82280: Em Cachoeira de o Sul , o levante se desenrolou sem qualquer contratempo : em o *entardecer* de o dia 4 de outubro , as unidades federais de a cidade já se achavam reorganizadas por os revolucionários , prontas para seguir para o norte de o estado , de onde rumariam para São Paulo e Rio .
337330: Em o segundo dia de luta , uma companhia de o 5º RI e todo o 6º RI , comandado por o major Raul Cabral Velho , aderiram a 

# Filtro D

In [14]:
def find_se(sentences, list_tokens):
    se = {"a": {}}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "Gender=Unsp" in token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["a"]:
                    se["a"][sent_id] = []
                se["a"][sent_id].append(sentence.map_token_id[token.head_token.id])

    return se
                
root_se = {}
subordinate_se = {}
for corpus in corpora:
    root_se[corpus] = find_se(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_se[corpus] = find_se(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_se[corpus]["a"]:
        for token in root_se[corpus]["a"][sentence]:
            if token in root_without_subject[corpus][sentence]:
                root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_se[corpus]["a"]:
        for token in subordinate_se[corpus]["a"][sentence]:
            if token in subordinate_without_subject[corpus][sentence]:
                subordinate_without_subject[corpus][sentence].remove(token)

In [15]:
for corpus in corpora:
    for filtro in ["a"]:
        print("\n{} - orações principais com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in root_se[corpus][filtro].values()])))
        print("{} - orações subordinadas com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in subordinate_se[corpus][filtro].values()])))

for corpus in corpora:
    for filtro in ["a"]:
        i = 0
        ii = 0
        print("\n=== {} exemplos de se (filtro {}) como filho de oração principal em {} ===".format(max_examples, filtro, corpus))
        for sent_id, tokens in root_se[corpus][filtro].items():
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
        print("\n=== {} exemplos de se (filtro {}) como filho de oração subordinada em {} ===".format(max_examples, filtro, corpus))
        for sent_id, tokens in subordinate_se[corpus][filtro].items():
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break


dhbb - orações principais com sujeito oculto e se (filtro a): 59
dhbb - orações subordinadas com sujeito oculto e se (filtro a): 757

obras - orações principais com sujeito oculto e se (filtro a): 215
obras - orações subordinadas com sujeito oculto e se (filtro a): 155

bosque - orações principais com sujeito oculto e se (filtro a): 31
bosque - orações subordinadas com sujeito oculto e se (filtro a): 5

=== 20 exemplos de se (filtro a) como filho de oração principal em dhbb ===
4288: Em o início de 1966 , a o se *formalizar* a criação de dois novos partidos políticos , a Aliança Renovadora Nacional ( Arena ) , de apoio a o governo , e o Movimento Democrático Brasileiro ( MDB ) , de oposição , filiou se a a Arena , tendo sido o primeiro presidente de esse partido em a Guanabara .
13465: *Recomendava* ainda que se de esse « preferência a o regime estatal » para a exploração de a indústria siderúrgica , propondo a criação de o Conselho Nacional de Minas e Metalurgia e de o Conselho Nacio

# Contagem final

In [16]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        if len(tokens):
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        if len(tokens):
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break


dhbb - orações principais com sujeito oculto final: 193381 / 480218
dhbb - orações subordinadas com sujeito oculto final: 188759 / 341133

obras - orações principais com sujeito oculto final: 123302 / 353662
obras - orações subordinadas com sujeito oculto final: 151364 / 316297

bosque - orações principais com sujeito oculto final: 1476 / 9364
bosque - orações subordinadas com sujeito oculto final: 2462 / 6842

=== 20 exemplos de sujeito oculto na oração principal em dhbb ===
2: *Estudou* em o Ginásio Diocesano de São Paulo e bacharelou se em 1910 por a Faculdade de Ciências Jurídicas e Sociais .
3: Dedicando se a a advocacia , foi *promotor* público em Cunha ( SP ) e depois delegado de polícia em o Rio de Janeiro , então Distrito Federal .
4: *Iniciou* sua vida política como deputado federal por o Distrito Federal , exercendo o mandato de 1927 a 1929 .
6: Ligado a o governo federal , *encontrava* se a o lado de o presidente Washington Luís , em o palácio Guanabara , em o momento de s

# Pickle Bosque para posterior reconstituição

In [17]:
import pickle

for corpus in path:
    for sentence in list(root_without_subject[corpus].keys()):
        if not root_without_subject[corpus][sentence]:
            del root_without_subject[corpus][sentence]
        
    for sentence in list(subordinate_without_subject[corpus].keys()):
        if not subordinate_without_subject[corpus][sentence]:
            del subordinate_without_subject[corpus][sentence]

    if corpus == "bosque":
        with open("{}_root.p".format(corpus), "wb") as f:
            pickle.dump(root_without_subject[corpus], f)
        with open("{}_subordinate.p".format(corpus), "wb") as f:
            pickle.dump(subordinate_without_subject[corpus], f)

# Reconstituição

In [18]:
import estrutura_ud
import pickle

with open("bosque_root.p", "rb") as f:
    root = pickle.load(f)
with open("bosque_subordinate.p", "rb") as f:
    subordinate = pickle.load(f)
    
len_root = sum([len(root[x]) for x in root])
len_subordinate = sum([len(subordinate[x]) for x in subordinate])
    
bosque = estrutura_ud.Corpus(recursivo=True)
bosque.load("bosque-ud-2.6.conllu")

dic_pronouns = {
    '1Sing': ['eu', 'eu'],
    '1SingMale': ['eu', 'eu'],
    '1SingFem': ['eu', 'eu'],
    '2Sing': ['tu', 'tu'],
    '2SingMale': ['tu', 'tu'],
    '2SingFem': ['tu', 'tu'],
    '3SingMale': ['ele', 'ele'],
    '3SingFem': ['ela', 'ela'],
    '1Plur': ['nós', 'nós'],
    '1PlurMale': ['nós', 'nós'],
    '1PlurFem': ['nós', 'nós'],
    '2Plur': ['vós', 'vós'],
    '2PlurMale': ['vós', 'vós'],
    '2PlurFem': ['vós', 'vós'],
    '3PlurMale': ['eles', 'eles'],
    '3PlurFem': ['elas', 'elas']
}

bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].deprel = "advcl"
bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].dephead = "13"            

In [19]:
def find_feats(feat, token):
    if not feat in token.feats:
        return ""
    return token.feats.split(feat + "=")[1].split("|")[0]

In [20]:
def insert_phrase(sentence, first_token_id, phrase):
    sintagmas_anteriores = []
    for t, token in enumerate(sentence.tokens):
        if t < sentence.map_token_id[first_token_id]:
            if token.dephead == first_token_id and (token.upos == "ADV" or (token.lemma == "se" and token.upos == "PRON") or token.deprel == "cop" or token.upos == "AUX"):
                is_sintagma = True
                for _token in sentence.tokens:
                    if _token.upos == "PUNCT" and _token.dephead == token.id:
                        is_sintagma = False
                        break
                if is_sintagma:
                    sintagmas_anteriores.append(sentence.tokens[t].id)
    who_head = first_token_id
    first_token_id = min(sintagmas_anteriores) if sintagmas_anteriores else first_token_id

    for new_token in reversed(phrase):
        token = estrutura_ud.Token()
        token.build(new_token.to_str())
        token.dephead = str(int(token.dephead) - int(token.id) + int(who_head.split("-")[0]))
        token.id = first_token_id.split("-")[0]
        if first_token_id == "1":
            if token.word != "<SUBJ>":
                token.word = token.word.title()
            sentence.tokens[0].word = sentence.tokens[0].word.lower()
        token.misc = "|".join(sorted([x for x in token.misc.split("|") + ["<SUBJ>"] if x != "_"]))
        sentence.tokens.insert(sentence.map_token_id[first_token_id], token)
        
        t_first_token_id = sentence.map_token_id[first_token_id]
        for t, token in enumerate(sentence.tokens):
            if t > t_first_token_id:
                token.id = str(int(token.id)+1) if not '-' in token.id else str(int(token.id.split("-")[0])+1) + "-" + str(int(token.id.split("-")[1])+1)
            sentence.map_token_id[token.id] = t
        for t, token in enumerate(sentence.tokens):
            if token.dephead not in ["0", "_"] and sentence.map_token_id[token.dephead] >= sentence.map_token_id[first_token_id]:
                token.dephead = str(int(token.dephead)+1)
        
    sentence.refresh_map_token_id()
                      
    sentence.metadados['text'] = " ".join([x.word for x in sentence.tokens if not '-' in x.id])
    return sentence

In [21]:
#ADVCL à esquerda com mesmo PESSNUM (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del root[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_root))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 13 / 1476

CP40-3:
Quando o povo suíço recusou, em 92, a adesão ao Espaço Económico Europeu, como já fizera com a ONU, cometeu um grave engano.
Quando o povo suíço recusou , em 92 , a adesão a o Espaço Económico Europeu , como já fizera com a ONU , o povo suíço cometeu um grave engano .

CP88-5:
Só quando Moniz Pereira lhe surge na frente, se compenetra de que era mesmo verdade.
Só quando Moniz Pereira lhe surge em a frente , Moniz Pereira se compenetra de que era mesmo verdade .

CP94-5:
Se os chefes militares, actualmente, despacham com o ministro de 15 em 15 dias, farão o mesmo com o primeiro-ministro e o secretário de Estado governa o Ministério.
Se os chefes militares , actualmente , despacham com o ministro de 15 em 15 dias , os chefes militares farão o mesmo com o primeiro-ministro e o secretário de Estado governa o Ministério .

CP493-7:
Como o vírus tem uma propensão para infectar as células humanas, consegue em princípio fazer penetrar o gene dentro das célula

In [22]:
#ADVCL à esquerda com mesmo PESSNUM (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del subordinate[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_subordinate))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 3 / 2462

CP636-2:
Os dirigentes da Fenprof avisam no entanto desde já que se Couto dos Santos insistir nalgumas das directrizes dos seus antecessores arranjará lenha para se queimar.
Os dirigentes de a Fenprof avisam em o entanto desde já que se Couto de os Santos insistir em algumas de as directrizes de os seus antecessores Couto Santos arranjará lenha para se queimar .

CF619-5:
Cabe perguntar: Se o Supremo não está de acordo com a suposta falta de regra das duas moedas, que existem de fato desde a emissão da última medida provisória, por que não a proclamou imediatamente inconstitucional e se serviu dela privadamente?
Cabe perguntar : Se o Supremo não está de acordo com a suposta falta de regra de as duas moedas , que existem de fato desde a emissão de a última medida provisória , por que o Supremo não a proclamou imediatamente inconstitucional e se serviu de ela privadamente ?

CF740-3:
Notamos ainda que, quando um passageiro brasileiro se inscreve num programa de 

In [23]:
#DESINÊNCIA (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for i, t in enumerate(tokens):
        has_aux = False
        for _t, _token in enumerate(sentence.tokens):
            if _token.dephead == sentence.tokens[t].id and _token.upos == "AUX":
                person = find_feats("Person", sentence.tokens[_t])
                number = find_feats("Number", sentence.tokens[_t])
                gender = find_feats("Gender", sentence.tokens[_t])
                has_aux = True
                break
        if not has_aux:
            person = find_feats("Person", sentence.tokens[t])
            number = find_feats("Number", sentence.tokens[t])
            gender = find_feats("Gender", sentence.tokens[t])
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'PRON', '_', feats, "1", 'nsubj', '_', '_']
        else:
            pronoun = ["1", "<SUBJ>", "<SUBJ>", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun))
    
        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), [new_token])
    del root[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_root))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 1463 / 1476

CF896-2:
Dos fumantes, 18,1 milhões são homens e 12,5 milhões, mulheres.
De os fumantes , 18,1 milhões <SUBJ> são homens e 12,5 milhões , mulheres .

CF899-2:
Na sua administração, executou um plano viário com largas avenidas interligando os bairros, que provocaram um boom imobiliário.
Em a sua administração , <SUBJ> executou um plano viário com largas avenidas interligando os bairros , que provocaram um boom imobiliário .

CF902-3:
Quem não quiser beber durante o jantar, poderá levar o champanhe para casa.
Quem não quiser beber durante o jantar , <SUBJ> poderá levar o champanhe para casa .

CF904-6:
É o que confirmam as estatísticas do Campeonato Brasileiro, encerrado ontem com o novo título palmeirense.
<SUBJ> é o que confirmam as estatísticas de o Campeonato Brasileiro , encerrado ontem com o novo título palmeirense .

CF908-4:
Diariamente, está promovendo desfiles de moda para seus consumidores.
Diariamente , <SUBJ> está promovendo desfiles de moda para

In [24]:
#DESINÊNCIA (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for i, t in enumerate(tokens):
        has_aux = False
        for _t, _token in enumerate(sentence.tokens):
            if _token.dephead == sentence.tokens[t].id and _token.upos == "AUX":
                person = find_feats("Person", sentence.tokens[_t])
                number = find_feats("Number", sentence.tokens[_t])
                gender = find_feats("Gender", sentence.tokens[_t])
                has_aux = True
                break
        if not has_aux:
            person = find_feats("Person", sentence.tokens[t])
            number = find_feats("Number", sentence.tokens[t])
            gender = find_feats("Gender", sentence.tokens[t])
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'PRON', '_', feats, "1", 'nsubj', '_', '_']
        else:
            pronoun = ["1", "<SUBJ>", "<SUBJ>", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun)) 

        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], bosque.sentences[sent_id].tokens[t+i].id, [new_token])
    del subordinate[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_subordinate))
print("\n" + "\n\n".join(reconstituidos[:20]))

Reconstituídos: 2027 / 2462

CF893-4:
Para construir o ambiente futurista do filme, Godard recorreu quase que totalmente à fotografia.
Para <SUBJ> construir o ambiente futurista de o filme , Godard recorreu quase que totalmente a a fotografia .

CF895-2:
O problema é político porque envolve, por exemplo, a gratuidade da educação, da saúde, da previdência mínima.
O problema é político porque <SUBJ> envolve , por exemplo , a gratuidade de a educação , de a saúde , de a previdência mínima .

CF895-3:
Excetuada a educação básica de primeiro grau, que deve ser gratuita e obrigatória, para todo o resto é preciso discriminar quem pode pagar.
<SUBJ> excetuada a educação básica de primeiro grau , que deve ser gratuita e obrigatória , para todo o resto é preciso discriminar quem pode pagar .

CF897-1:
PF prende dois por cobrar ágio
PF prende dois por <SUBJ> cobrar ágio

CF897-3:
A PF (Polícia Federal) indiciou ontem duas pessoas por vender carro com ágio.
A PF ( Polícia Federal ) indiciou ontem 

In [25]:
# Salvar modificado
bosque.save("bosque-ud-2.6-desocultado.conllu")

# Avaliação

# Debugagem

In [ ]:
print(corpora['obras'].sentences["540"].to_str())

In [ ]:
#root_nominals["obras"]
root_without_subject["obras"]